In [8]:
import sqlite3, csv, os
from sqlite3 import Error
db_filename = "cu_booking.db"

In [9]:
os.remove(db_filename)

In [10]:
def connect_db(db_filename):
    try:
        conn = sqlite3.connect(db_filename)
        return conn
    except Error as e:
        print (e)

In [11]:
conn = connect_db(db_filename)
conn.close()

In [12]:
## Task 1.1
##8
def create_tables(conn):
    ##1
    sql_student = '''
                    CREATE TABLE student (
                        id TEXT NOT NULL,
                        name TEXT NOT NULL,
                        contact INTEGER NOT NULL,
                        PRIMARY KEY (id)
                    );
                '''
    ##3
    sql_cubicle = '''
                    CREATE TABLE cubicle (
                        cubicle_no TEXT NOT NULL,
                        maintenance INTEGER DEFAULT 0,
                        PRIMARY KEY (cubicle_no),
                        CHECK (maintenance in (1,0))
                    )
                '''
    ##3
    sql_booking = '''
                    CREATE TABLE booking (
                        cubicle_no TEXT NOT NULL,
                        date TEXT NOT NULL,
                        student_id TEXT NOT NULL,
                        PRIMARY KEY (cubicle_no, date),
                        FOREIGN KEY (cubicle_no) REFERENCES cubicle (cubicle_no),
                        FOREIGN KEY (student_id) REFERENCES student (id)
                    )
                '''
    for table in [sql_student,sql_cubicle,sql_booking]:
        try:
            conn.execute(table) ##1
        except Error as e:
            print(e)


In [13]:
conn = connect_db(db_filename)
create_tables(conn)
conn.close()

In [14]:
## Task 1.2
##5
def import_csv(conn, csv_file):
    with open (csv_file,'r') as csvfile:
        table = csv_file[:-4]
        csvreader = csv.reader(csvfile)
        header = tuple(next(csvreader)) #1
        for row in csvreader:
            sql = f"INSERT INTO {table} {header} VALUES {tuple(row)}" # 3
            try:
                conn.execute(sql) 
                conn.commit() #1
            except Error as e:
                print(e)
    csvfile.close()
       

In [15]:
conn = connect_db(db_filename)

csv_files = ["student.csv", "cubicle.csv", "booking.csv"]
for csv_file in csv_files:
    import_csv(conn,csv_file)
conn.close()

In [16]:
## Task 1.3
##2
def update_maintenance(conn, cu, new):
    sql = "UPDATE cubicle SET maintenance = ? WHERE cubicle_no = ?" #1
    try:
        conn.execute(sql,(new,cu))#1
        conn.commit()
    except Error as e:
        print(e)

In [17]:
conn = connect_db(db_filename)
update_maintenance(conn, "c01", 0)
conn.close()

In [18]:
## Task 1.4
## 3
def not_avail(conn,date):
    sql = '''
        SELECT cubicle_no FROM cubicle
        WHERE cubicle_no NOT IN (
        (SELECT cubicle.cubicle_no
            FROM cubicle 
        LEFT JOIN booking
            ON cubicle.cubicle_no = booking.cubicle_no
        WHERE date = ? OR maintenance = 1))
    ''' #2
    cur = conn.execute(sql,(date,))
    unavail_list = [i[0] for i in cur.fetchall()] #1
    return unavail_list

In [19]:
conn = connect_db(db_filename)
lst = not_avail(conn,"17112024")
print(lst)
conn.close()

['c01', 'c03', 'c04', 'c05', 'c06', 'c07', 'c08', 'c09', 'c10', 'c11', 'c12']
